# Exercício 01 - Regressão Linear
Nome: Antônio Augusto Carnevalli

Nº USP: 13682909

Exercício feito individualmente.

In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
cars_data = pd.read_csv("vehicles_cleaned_train.csv")
cars_data.head()
#cars_data.info()

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,10400,2011,excellent,4 cylinders,gas,81300,automatic,mid-size,sedan
1,6900,2007,excellent,6 cylinders,gas,79000,automatic,full-size,sedan
2,18900,2018,like new,4 cylinders,gas,5000,automatic,full-size,sedan
3,6000,2010,excellent,4 cylinders,gas,97600,automatic,mid-size,hatchback
4,19995,2013,good,6 cylinders,gas,95782,automatic,mid-size,sedan


In [152]:
# Criando variáveis dummy para as entradas de tipo categóricas utilizando one-hot encoding
# Usaremos o drop_first=True para evitar problemas de multicolinearidade
# Após isso, iremos correlacionar todas as variáveis com price e ver se há algo que possa ser descartado
x1 = pd.get_dummies(cars_data['condition'], drop_first=True)
x2 = pd.get_dummies(cars_data['type'], drop_first=True)
x3 = pd.get_dummies(cars_data['fuel'], drop_first=True)
x4 = pd.get_dummies(cars_data['transmission'], drop_first=True)
x5 = pd.get_dummies(cars_data['cylinders'], drop_first=True)
x6 = pd.get_dummies(cars_data['size'], drop_first=True)

# Criando variáveis de entrada para os tipos numéricos
x7 = cars_data['odometer']
x8 = cars_data['year']

# Criando o vetor de saída desejada d
#d = cars_data['price']

# Criaremos uma nova data frame com as variáveis dummy concatenadas
lista_dummies = [x1, x2, x3, x4, x5, x6]
cars_data_merged = cars_data
for i in range(len(lista_dummies)):
  cars_data_merged = pd.concat(([cars_data_merged, lista_dummies[i]]), axis=1)
#print(cars_data_merged)

# Com a lista já com as variáveis dummy, iremos remover as variáveis categóricas originais
lista_categoricas = ['condition', 'type', 'fuel', 'transmission', 'cylinders', 'size']
cars_data_merged = cars_data_merged.drop((lista_categoricas), axis=1)
#print(cars_data_merged)

# Agora que temos a nova data frame sem variáveis categóricas, podemos estudar a correlação das variáveis com a variável 'price'
print(cars_data_merged.corr())

# Com essa table de correlação, vemos que os fatores que mais influenciaram o preço foram a idade do carro,
# a milhagem e condição. E as que menos influenciaram foram transmissião, cilindragem, combustível e tamanho

                price      year  odometer      fair      good  like new  \
price        1.000000  0.520877 -0.380056 -0.096375 -0.199158  0.154598   
year         0.520877  1.000000 -0.377901 -0.097870 -0.199903  0.160445   
odometer    -0.380056 -0.377901  1.000000  0.035685  0.126310 -0.208011   
fair        -0.096375 -0.097870  0.035685  1.000000 -0.039550 -0.046061   
good        -0.199158 -0.199903  0.126310 -0.039550  1.000000 -0.251335   
like new     0.154598  0.160445 -0.208011 -0.046061 -0.251335  1.000000   
new          0.016947  0.077117 -0.108341 -0.007488 -0.040857 -0.047583   
salvage     -0.042552  0.017399 -0.009797 -0.003960 -0.021608 -0.025165   
hatchback   -0.048522  0.009983 -0.017778  0.007844  0.040872 -0.010280   
sedan       -0.047014  0.057433  0.019159 -0.000753 -0.030734  0.013096   
wagon        0.042593  0.001997 -0.004672  0.009966 -0.017877 -0.012812   
gas          0.012198  0.012343 -0.053493  0.009285  0.006975  0.016460   
manual       0.042481 -0.

In [147]:
from numpy.linalg import inv
# Para a regressão linear, utilizaremos, na nomenclatura original da cars_data, as colunas year, odometer, condition e type
dfRegressao = pd.read_csv('vehicles_cleaned_train.csv')

# Fazendo o tratamento inicial
lista_removidos = ['fuel', 'transmission', 'cylinders', 'size']
dfRegressao = dfRegressao.drop((lista_removidos), axis=1)
dummies_condition = pd.get_dummies(cars_data['condition'], drop_first=True)
dummies_type = pd.get_dummies(cars_data['type'], drop_first=True)
dfRegressaoMerged = dfRegressao
dfRegressaoMerged = pd.concat((dfRegressaoMerged, dummies_condition), axis=1)
dfRegressaoMerged = pd.concat((dfRegressaoMerged, dummies_type), axis=1)
dfRegressaoMerged = dfRegressaoMerged.drop(['condition', 'type'], axis=1)

# Nosso data frame de operação será, portanto, dfRegressaoMerged. 
# Ele já possui os dados adaptados para serem utilizados como valores de entrada do vetor X
# Converteremos ele para formato numpy agora
df_np = dfRegressaoMerged.to_numpy()
x_treino, y_treino = df_np[:, 1:], df_np[:, 0]

# Convertendo para dados tipo float
x_treino = x_treino.astype(float)
y_treino = y_treino.astype(float)

# Criaremos em seguida a função que retorna os melhores 'ômegas' para a previsão baseada nas entradas, e o erro também.
def PesosOtimizadosRegressao(X, d):
  (n, p_MenosUm) = X.shape

  'A primeira coluna da matriz X deve ser preenchida com 1s devido ao bias'
  p = p_MenosUm + 1
  matriz_X = np.ones(shape=(n, p))
  matriz_X[:, 1:] = X

  # Cálculo do array wo
  XT = np.transpose(matriz_X)
  R = np.matmul(XT, matriz_X)
  p = np.matmul(XT, d)
  wo = np.linalg.solve(R, p)

  # Cálculo do array erro
  e = np.subtract(d, np.matmul(matriz_X, wo))

  return wo, e


# Função que realiza previsão utilizando ômegas desejados
def Previsao(w, X_entradas):

  (n, p_MenosUm) = X_entradas.shape
  p = p_MenosUm + 1

  'A primeira coluna da matriz X deve ser preenchida com 1s devido ao bias'
  matriz_X = np.ones(shape=(n, p))
  matriz_X[:, 1:] = X_entradas
  previsao = np.dot(matriz_X, w)

  return previsao


melhor_wo = PesosOtimizadosRegressao(x_treino, y_treino)

## **Partiremos agora para o data frame de testes**

In [161]:
# Importando o data frame e realizando o mesmo tratamento de antes
dfTeste = pd.read_csv('vehicles_cleaned_test.csv')
lista_removidos = ['fuel', 'transmission', 'cylinders', 'size']
dfTeste = dfTeste.drop((lista_removidos), axis=1)
dummies_condition = pd.get_dummies(dfTeste['condition'], drop_first=True)
dummies_type = pd.get_dummies(dfTeste['type'], drop_first=True)
dfTesteMerged = dfTeste
dfTesteMerged = pd.concat((dfTesteMerged, dummies_condition), axis=1)
dfTesteMerged = pd.concat((dfTesteMerged, dummies_type), axis=1)
dfTesteMerged = dfTesteMerged.drop(['condition', 'type'], axis=1)
dfTeste_np = dfTesteMerged.to_numpy()
x_teste, y_teste = dfTeste_np[:, 1:], dfTeste_np[:, 0]
x_teste = x_teste.astype(float)
y_teste = y_teste.astype(float)

# Com as variáveis de teste prontas, iremos aplica-lás nas funções anteriores



# Função que calcula o MSE
def mse(valor_real, valor_previsto):
    return ((valor_real - valor_previsto) ** 2).mean()


# Módulo do erro quadrático médio do modelo com os dados de treinamento
print('O erro quadrático médio do modelo com os dados de treinamento foi de aproximadamente: ' +str(int(mse(y_treino,Previsao(melhor_wo[0], x_treino)))))

# Módulo do erro quadrático médio do modelo com os dados de teste
print('O erro quadrático médio do modelo com os dados de teste foi de aproximadamente: ' +str(int(mse(y_teste,Previsao(melhor_wo[0], x_teste)))))
print('O MSE foi alto, mas pode ocorrer pela diferença entre valores de preço.')
print('No entanto, observa-se que a a diferença percentual do MSE de teste, para o MSE de treino foi baixa.')

O erro quadrático médio do modelo com os dados de treinamento foi de aproximadamente: 29595852
O erro quadrático médio do modelo com os dados de teste foi de aproximadamente: 28379042
O MSE foi alto, mas pode ocorrer pela diferença entre valores de preço.
No entanto, observa-se que a a diferença percentual do MSE de teste, para o MSE de treino foi baixa.
